See more: https://www.tensorflow.org/versions/r1.2/get_started/mnist/pros

In [1]:
# Load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


/home/hung/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
print(mnist)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5f76740518>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5f43346e10>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5f43346828>)


In [3]:
print(mnist.train)

In [10]:
mnist.train.images.shape

(55000, 784)

In [18]:
mnist.train.images.tolist

<function ndarray.tolist>

In [26]:
# setup environment
import tensorflow as tf
sess = tf.InteractiveSession()

In [20]:
# placeholder
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [48]:
print(x, y_, sep="\n")

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)


In [33]:
# Variables
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [49]:
print(W, b, sep="\n")

<tf.Variable 'Variable_2:0' shape=(784, 10) dtype=float32_ref>
<tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>


In [36]:
# initialize variables
sess.run(tf.global_variables_initializer())

In [34]:
print(W.shape[1])

10


In [37]:
# Predicted Class and Loss Function
y = tf.matmul(x, W) + b

In [43]:
y

<tf.Tensor 'add:0' shape=(?, 10) dtype=float32>

In [39]:
# loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [42]:
cross_entropy

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [40]:
# Train the Model
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [41]:
train_step

<tf.Operation 'GradientDescent' type=NoOp>

In [50]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})


In [51]:
# Evaluate the Model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [55]:
correct_prediction.get_shape()

TensorShape([Dimension(None)])

In [56]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [57]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9159


In [71]:
train_step.type

'NoOp'

### Build a Multilayer Convolutional Network

Getting 92% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.
#### Weight Initialization
To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [73]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and Pooling
TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [74]:
def conv2d(x, W):
      return tf.nn.conv2d(
          x, 
          W, 
          strides=[1, 1, 1, 1], padding='SAME'
      )

def max_pool_2x2(x):
      return tf.nn.max_pool(
          x, 
          ksize=[1, 2, 2, 1],
          strides=[1, 2, 2, 1], 
          padding='SAME'
      )

#### First Convolutional Layer
We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [75]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [76]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool. The max_pool_2x2 method will reduce the image size to 14x14.

In [77]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### Second Convolutional Layer
In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch

In [78]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#### Densely Connected Layer
Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [79]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


##### Dropout
To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.1

In [80]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

##### Readout Layer

Finally, we add a layer, just like for the one layer softmax regression above.

In [81]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#### Train and Evaluate the Model
How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above.

The differences are that:

    * We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.

    * We will include the additional parameter keep_prob in feed_dict to control the dropout rate.

    * We will add logging to every 100th iteration in the training process.

We will also use tf.Session rather than tf.InteractiveSession. This better separates the process of creating the graph (model sepecification) and the process of evaluating the graph (model fitting). It generally makes for cleaner code. The tf.Session is created within a with block so that it is automatically destroyed once the block is exited.

Feel free to run this code. Be aware that it does 20,000 training iterations and may take a while (possibly up to half an hour), depending on your processor.

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
              train_accuracy = accuracy.eval(feed_dict={
                                  x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


step 0, training accuracy 0.08
step 1, training accuracy 0.08
step 2, training accuracy 0.08
step 3, training accuracy 0.08
step 4, training accuracy 0.08
step 5, training accuracy 0.08
step 6, training accuracy 0.08
step 7, training accuracy 0.08
step 8, training accuracy 0.08
step 9, training accuracy 0.08
step 10, training accuracy 0.08
step 11, training accuracy 0.08
step 12, training accuracy 0.08
step 13, training accuracy 0.08
step 14, training accuracy 0.08
step 15, training accuracy 0.08
step 16, training accuracy 0.08
step 17, training accuracy 0.08
step 18, training accuracy 0.08
step 19, training accuracy 0.08
step 20, training accuracy 0.08
step 21, training accuracy 0.08
step 22, training accuracy 0.08
step 23, training accuracy 0.08
step 24, training accuracy 0.08
step 25, training accuracy 0.08
step 26, training accuracy 0.08
step 27, training accuracy 0.08
step 28, training accuracy 0.08
step 29, training accuracy 0.08
step 30, training accuracy 0.08
step 31, training 

step 252, training accuracy 0.92
step 253, training accuracy 0.92
step 254, training accuracy 0.92
step 255, training accuracy 0.92
step 256, training accuracy 0.92
step 257, training accuracy 0.92
step 258, training accuracy 0.92
step 259, training accuracy 0.92
step 260, training accuracy 0.92
step 261, training accuracy 0.92
step 262, training accuracy 0.92
step 263, training accuracy 0.92
step 264, training accuracy 0.92
step 265, training accuracy 0.92
step 266, training accuracy 0.92
step 267, training accuracy 0.92
step 268, training accuracy 0.92
step 269, training accuracy 0.92
step 270, training accuracy 0.92
step 271, training accuracy 0.92
step 272, training accuracy 0.92
step 273, training accuracy 0.92
step 274, training accuracy 0.92
step 275, training accuracy 0.92
step 276, training accuracy 0.92
step 277, training accuracy 0.92
step 278, training accuracy 0.92
step 279, training accuracy 0.92
step 280, training accuracy 0.92
step 281, training accuracy 0.92
step 282, 

step 504, training accuracy 0.96
step 505, training accuracy 0.96
step 506, training accuracy 0.96
step 507, training accuracy 0.96
step 508, training accuracy 0.96
step 509, training accuracy 0.96
step 510, training accuracy 0.96
step 511, training accuracy 0.96
step 512, training accuracy 0.96
step 513, training accuracy 0.96
step 514, training accuracy 0.96
step 515, training accuracy 0.96
step 516, training accuracy 0.96
step 517, training accuracy 0.96
step 518, training accuracy 0.96
step 519, training accuracy 0.96
step 520, training accuracy 0.96
step 521, training accuracy 0.96
step 522, training accuracy 0.96
step 523, training accuracy 0.96
step 524, training accuracy 0.96
step 525, training accuracy 0.96
step 526, training accuracy 0.96
step 527, training accuracy 0.96
step 528, training accuracy 0.96
step 529, training accuracy 0.96
step 530, training accuracy 0.96
step 531, training accuracy 0.96
step 532, training accuracy 0.96
step 533, training accuracy 0.96
step 534, 

step 754, training accuracy 0.98
step 755, training accuracy 0.98
step 756, training accuracy 0.98
step 757, training accuracy 0.98
step 758, training accuracy 0.98
step 759, training accuracy 0.98
step 760, training accuracy 0.98
step 761, training accuracy 0.98
step 762, training accuracy 0.98
step 763, training accuracy 0.98
step 764, training accuracy 0.98
step 765, training accuracy 0.98
step 766, training accuracy 0.98
step 767, training accuracy 0.98
step 768, training accuracy 0.98
step 769, training accuracy 0.98
step 770, training accuracy 0.98
step 771, training accuracy 0.98
step 772, training accuracy 0.98
step 773, training accuracy 0.98
step 774, training accuracy 0.98
step 775, training accuracy 0.98
step 776, training accuracy 0.98
step 777, training accuracy 0.98
step 778, training accuracy 0.98
step 779, training accuracy 0.98
step 780, training accuracy 0.98
step 781, training accuracy 0.98
step 782, training accuracy 0.98
step 783, training accuracy 0.98
step 784, 

step 1003, training accuracy 0.96
step 1004, training accuracy 0.96
step 1005, training accuracy 0.96
step 1006, training accuracy 0.96
step 1007, training accuracy 0.96
step 1008, training accuracy 0.96
step 1009, training accuracy 0.96
step 1010, training accuracy 0.96
step 1011, training accuracy 0.96
step 1012, training accuracy 0.96
step 1013, training accuracy 0.96
step 1014, training accuracy 0.96
step 1015, training accuracy 0.96
step 1016, training accuracy 0.96
step 1017, training accuracy 0.96
step 1018, training accuracy 0.96
step 1019, training accuracy 0.96
step 1020, training accuracy 0.96
step 1021, training accuracy 0.96
step 1022, training accuracy 0.96
step 1023, training accuracy 0.96
step 1024, training accuracy 0.96
step 1025, training accuracy 0.96
step 1026, training accuracy 0.96
step 1027, training accuracy 0.96
step 1028, training accuracy 0.96
step 1029, training accuracy 0.96
step 1030, training accuracy 0.96
step 1031, training accuracy 0.96
step 1032, tra

...

step 18866, training accuracy 1
step 18867, training accuracy 1
step 18868, training accuracy 1
step 18869, training accuracy 1
step 18870, training accuracy 1
step 18871, training accuracy 1
step 18872, training accuracy 1
step 18873, training accuracy 1
step 18874, training accuracy 1
step 18875, training accuracy 1
step 18876, training accuracy 1
step 18877, training accuracy 1
step 18878, training accuracy 1
step 18879, training accuracy 1
step 18880, training accuracy 1
step 18881, training accuracy 1
step 18882, training accuracy 1
step 18883, training accuracy 1
step 18884, training accuracy 1
step 18885, training accuracy 1
step 18886, training accuracy 1
step 18887, training accuracy 1
step 18888, training accuracy 1
step 18889, training accuracy 1
step 18890, training accuracy 1
step 18891, training accuracy 1
step 18892, training accuracy 1
step 18893, training accuracy 1
step 18894, training accuracy 1
step 18895, training accuracy 1
step 18896, training accuracy 1
step 188

step 19124, training accuracy 1
step 19125, training accuracy 1
step 19126, training accuracy 1
step 19127, training accuracy 1
step 19128, training accuracy 1
step 19129, training accuracy 1
step 19130, training accuracy 1
step 19131, training accuracy 1
step 19132, training accuracy 1
step 19133, training accuracy 1
step 19134, training accuracy 1
step 19135, training accuracy 1
step 19136, training accuracy 1
step 19137, training accuracy 1
step 19138, training accuracy 1
step 19139, training accuracy 1
step 19140, training accuracy 1
step 19141, training accuracy 1
step 19142, training accuracy 1
step 19143, training accuracy 1
step 19144, training accuracy 1
step 19145, training accuracy 1
step 19146, training accuracy 1
step 19147, training accuracy 1
step 19148, training accuracy 1
step 19149, training accuracy 1
step 19150, training accuracy 1
step 19151, training accuracy 1
step 19152, training accuracy 1
step 19153, training accuracy 1
step 19154, training accuracy 1
step 191

step 19382, training accuracy 1
step 19383, training accuracy 1
step 19384, training accuracy 1
step 19385, training accuracy 1
step 19386, training accuracy 1
step 19387, training accuracy 1
step 19388, training accuracy 1
step 19389, training accuracy 1
step 19390, training accuracy 1
step 19391, training accuracy 1
step 19392, training accuracy 1
step 19393, training accuracy 1
step 19394, training accuracy 1
step 19395, training accuracy 1
step 19396, training accuracy 1
step 19397, training accuracy 1
step 19398, training accuracy 1
step 19399, training accuracy 1
step 19400, training accuracy 1
step 19401, training accuracy 1
step 19402, training accuracy 1
step 19403, training accuracy 1
step 19404, training accuracy 1
step 19405, training accuracy 1
step 19406, training accuracy 1
step 19407, training accuracy 1
step 19408, training accuracy 1
step 19409, training accuracy 1
step 19410, training accuracy 1
step 19411, training accuracy 1
step 19412, training accuracy 1
step 194

step 19639, training accuracy 1
step 19640, training accuracy 1
step 19641, training accuracy 1
step 19642, training accuracy 1
step 19643, training accuracy 1
step 19644, training accuracy 1
step 19645, training accuracy 1
step 19646, training accuracy 1
step 19647, training accuracy 1
step 19648, training accuracy 1
step 19649, training accuracy 1
step 19650, training accuracy 1
step 19651, training accuracy 1
step 19652, training accuracy 1
step 19653, training accuracy 1
step 19654, training accuracy 1
step 19655, training accuracy 1
step 19656, training accuracy 1
step 19657, training accuracy 1
step 19658, training accuracy 1
step 19659, training accuracy 1
step 19660, training accuracy 1
step 19661, training accuracy 1
step 19662, training accuracy 1
step 19663, training accuracy 1
step 19664, training accuracy 1
step 19665, training accuracy 1
step 19666, training accuracy 1
step 19667, training accuracy 1
step 19668, training accuracy 1
step 19669, training accuracy 1
step 196

step 19896, training accuracy 1
step 19897, training accuracy 1
step 19898, training accuracy 1
step 19899, training accuracy 1
step 19900, training accuracy 1
step 19901, training accuracy 1
step 19902, training accuracy 1
step 19903, training accuracy 1
step 19904, training accuracy 1
step 19905, training accuracy 1
step 19906, training accuracy 1
step 19907, training accuracy 1
step 19908, training accuracy 1
step 19909, training accuracy 1
step 19910, training accuracy 1
step 19911, training accuracy 1
step 19912, training accuracy 1
step 19913, training accuracy 1
step 19914, training accuracy 1
step 19915, training accuracy 1
step 19916, training accuracy 1
step 19917, training accuracy 1
step 19918, training accuracy 1
step 19919, training accuracy 1
step 19920, training accuracy 1
step 19921, training accuracy 1
step 19922, training accuracy 1
step 19923, training accuracy 1
step 19924, training accuracy 1
step 19925, training accuracy 1
step 19926, training accuracy 1
step 199